In [19]:
import plotly.graph_objects as go

from apollo.calculations.average_directional_movement_index import (
    AverageDirectionalMovementIndexCalculator,
)
from apollo.calculations.average_true_range import AverageTrueRangeCalculator
from apollo.providers.price_data_enhancer import PriceDataEnhancer
from apollo.providers.price_data_provider import PriceDataProvider
from apollo.settings import FREQUENCY, TICKER
from apollo.utils.common import to_default_date_string

In [20]:
start_date = "1995-01-01"
end_date = "2024-09-01"

window_size = 14

price_data_provider = PriceDataProvider()
price_data_enhancer = PriceDataEnhancer()

dataframe = price_data_provider.get_price_data(
    ticker=str(TICKER),
    frequency=str(FREQUENCY),
    start_date=start_date,
    end_date=end_date,
    max_period=False,
)

dataframe = price_data_enhancer.enhance_price_data(
    price_dataframe=dataframe,
    additional_data_enhancers=["VIX", "SP500 Futures"],
)

dataframe["prev_close"] = dataframe["adj close"].shift(1)

atr_calculator = AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=window_size,
)
atr_calculator.calculate_average_true_range()

adx_calculator = AverageDirectionalMovementIndexCalculator(
    dataframe=dataframe,
    window_size=window_size,
)
adx_calculator.calculate_average_directional_movement_index()

dataframe = dataframe.loc["2020-01-01":"2022-01-01"]

dataframe

,ticker,adj close,adj high,adj low,adj open,adj volume,close,high,low,open,...,prev_high,mdm,pdm,pdi,mdi,dx,adx,adxr,prev_adx,prev_adxr
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,SPY,302.208649,302.227272,300.031879,300.971435,5.502510e+07,324.869995,324.890015,322.529999,323.540009,...,299.659784,0.506037,0.509547,0.273741,0.271855,0.345649,-2.061124,-4.667950,-2.246260,-5.110655
2020-01-03,SPY,299.920227,301.064438,298.701609,298.757421,7.228904e+07,322.410004,323.640015,321.100006,321.160004,...,302.227272,0.510077,0.507350,0.256372,0.257749,0.267990,-1.894758,-4.322856,-2.061124,-4.667950
2020-01-06,SPY,301.064423,301.148141,298.013193,298.134129,5.177175e+07,323.640015,323.730011,320.359985,320.489990,...,301.064438,0.501579,0.502526,0.243763,0.243304,0.094305,-1.752682,-3.980531,-1.894758,-4.322856
2020-01-07,SPY,300.217957,300.971453,299.762117,300.487707,3.767157e+07,322.730011,323.540009,322.239990,323.019989,...,301.148141,0.496802,0.490155,0.244186,0.247497,0.673494,-1.579384,-3.606637,-1.752682,-3.980531
2020-01-08,SPY,301.817932,303.055145,300.162098,300.413253,6.353200e+07,324.450012,325.779999,322.670013,322.940002,...,300.971453,0.483875,0.488826,0.236083,0.233691,0.509051,-1.430210,-3.282057,-1.579384,-3.606637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,SPY,458.288239,458.336239,453.246934,453.294934,5.455038e+07,477.260010,477.309998,472.010010,472.059998,...,453.419767,0.284034,0.282352,0.045698,0.045970,0.296928,36.963721,22.014483,39.784244,22.187549
2021-12-28,SPY,457.913666,459.776550,457.135867,458.729883,4.539536e+07,476.869995,478.809998,476.059998,477.720001,...,458.336239,0.356908,0.318876,0.053810,0.060228,5.627903,34.725448,22.610583,36.963721,22.014483
2021-12-29,SPY,458.499512,459.536567,457.001526,458.019387,5.233643e+07,477.480011,478.559998,475.920013,476.980011,...,459.776550,0.383956,0.346098,0.060892,0.067553,5.185671,32.615464,23.225997,34.725448,22.610583


In [21]:
value_1 = "adj close"
value_2 = "adx"
value_3 = "adxr"
value_4 = "mdi"

x = dataframe.index.to_numpy()

y1 = dataframe[value_1].to_numpy()
y2 = dataframe[value_2].to_numpy()
y3 = dataframe[value_3].to_numpy()
y4 = dataframe[value_4].to_numpy()

trace1 = go.Scatter(x=x, y=y1, name=value_1, yaxis="y1")
trace2 = go.Scatter(x=x, y=y2, name=value_2, yaxis="y2")
trace3 = go.Scatter(x=x, y=y3, name=value_3, yaxis="y2")
trace4 = go.Scatter(x=x, y=y4, name=value_4, yaxis="y2")

# Plot title
title = (
    f"{dataframe.iloc[0]["ticker"]} | VIX | Futures"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(
    data=[
        trace1,
        trace2,
        trace3,
        # trace4,
    ],
    layout=layout,
)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)